# OPIM 5894 Lecture 5. Data Aggregation
Jing Peng <br>
UConn School of Business <br>
September 29, 2017 <br>
Reference: https://pandas.pydata.org/pandas-docs/stable/groupby.html

* **GroupBy**
    * What is it?
    * How to Use?   <br><br>
* **Aggregation: computing a summary statistic (or statistics) about each group**
    * Compute group sums or means
    * Compute group sizes / counts    <br><br>
* **Transformation: perform some group-specific computations and return a like-indexed**
    * Standardizing data (zscore) within group
    * Filling NAs within groups with a value derived from each group    <br><br>
* **Filtration: discard some groups, according to a group-wise computation that evaluates True or False**
    * Discarding data that belongs to groups with only a few members
    * Filtering out data based on the group sum or mean    <br><br>
* **Apply: versatile, including things can't be done by methods listed above**

In [1]:
import numpy as np
import pandas as pd

## 1. What is GroupBy?

In [2]:
np.random.seed(0)
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                    'B' : ['one', 'one', 'two', 'three',
                           'two', 'two', 'one', 'three'],
                    'C' : np.random.randn(8),
                    'D' : np.random.randn(8)})
df

,A,B,C,D
0,foo,one,1.764052,-0.103219
1,bar,one,0.400157,0.410599
2,foo,two,0.978738,0.144044
3,bar,three,2.240893,1.454274
4,foo,two,1.867558,0.761038
5,bar,two,-0.977278,0.121675
6,foo,one,0.950088,0.443863
7,foo,three,-0.151357,0.333674


### 1.1 Some explorations

In [3]:
grouped = df.groupby('A')
grouped

In [4]:
# What's in there? A list of tuples
# What is done? 
# Sort data over the group indicator and creates a GroupBy object that is ready for operations on groups
list(grouped)

[('bar',      A      B         C         D
  1  bar    one  0.400157  0.410599
  3  bar  three  2.240893  1.454274
  5  bar    two -0.977278  0.121675), ('foo',      A      B         C         D
  0  foo    one  1.764052 -0.103219
  2  foo    two  0.978738  0.144044
  4  foo    two  1.867558  0.761038
  6  foo    one  0.950088  0.443863
  7  foo  three -0.151357  0.333674)]

In [7]:
# df.groupby?

### 1.2 Some methods on GroupBy objects

In [8]:
grouped.groups

{'bar': Int64Index([1, 3, 5], dtype='int64'),
 'foo': Int64Index([0, 2, 4, 6, 7], dtype='int64')}

In [11]:
grouped.size()

A
bar    3
foo    5
dtype: int64

In [12]:
grouped.count()

,B,C,D
A,,,
bar,3,3,3
foo,5,5,5


In [13]:
len(grouped)

2

In [14]:
# The first obs of each group
grouped.first()

,B,C,D
A,,,
bar,one,0.400157,0.410599
foo,one,1.764052,-0.103219


In [15]:
# The sum of columns for each group
grouped.sum()

,C,D
A,,
bar,1.663773,1.986547
foo,5.409080,1.579400


In [17]:
# Get obs from a certain group
grouped.get_group('bar')

,B,C,D
1,one,0.400157,0.410599
3,three,2.240893,1.454274
5,two,-0.977278,0.121675


### 1.3 Iterate through groups

In [19]:
# one key
grouped = df.groupby('A')
for (name, group) in grouped:
    print(name)
    print(group)

bar
     A      B         C         D
1  bar    one  0.400157  0.410599
3  bar  three  2.240893  1.454274
5  bar    two -0.977278  0.121675
foo
     A      B         C         D
0  foo    one  1.764052 -0.103219
2  foo    two  0.978738  0.144044
4  foo    two  1.867558  0.761038
6  foo    one  0.950088  0.443863
7  foo  three -0.151357  0.333674


In [26]:
# multiple keys
grouped = df.groupby(['A','B'])
for name, group in grouped:
    print(name)
    print(group)

('bar', 'one')
     A    B         C         D
1  bar  one  0.400157  0.410599
('bar', 'three')
     A      B         C         D
3  bar  three  2.240893  1.454274
('bar', 'two')
     A    B         C         D
5  bar  two -0.977278  0.121675
('foo', 'one')
     A    B         C         D
0  foo  one  1.764052 -0.103219
6  foo  one  0.950088  0.443863
('foo', 'three')
     A      B         C         D
7  foo  three -0.151357  0.333674
('foo', 'two')
     A    B         C         D
2  foo  two  0.978738  0.144044
4  foo  two  1.867558  0.761038


## 2. How to GroupBy?
### 2.1 GroupBy Names

In [ ]:
# one key
df.groupby('A').groups

In [ ]:
# multiple keys
df.groupby(['A','B']).groups

### 2.2 GroupBy Index

In [27]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
        ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
index = pd.MultiIndex.from_arrays(arrays, names=['first', 'second'])
df2 = pd.DataFrame({'A': [1, 1, 1, 1, 2, 2, 3, 3], 'B': np.arange(8)}, index=index)
df2

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [28]:
# By index level
df2.groupby(level=0).sum()

,A,B
first,,
bar,2,1
baz,2,5
foo,4,9
qux,6,13


In [29]:
# By index name
df2.groupby(level=['first', 'second']).sum()

A  B
first second      
bar   one     1  0
      two     1  1
baz   one     1  2
      two     1  3
foo   one     2  4
      two     2  5
qux   one     3  6
      two     3  7

In [ ]:
# By index names, just like by column names
df2.groupby(['first', 'second']).sum()

### 2.3 GroupBy Index and Names

In [30]:
# mixed: index and column name
df2.groupby([pd.Grouper(level=1), 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

In [31]:
# mixed: index name and column name
df2.groupby(['second', 'A']).sum()

B
second A   
one    1  2
       2  4
       3  6
two    1  4
       2  5
       3  7

### 2.4 GroupBy a function

In [38]:
# a function of column (or row) labels
def get_letter_type(letter):
    if letter.lower() in 'aeiou':
        return 'vowel'
    else:
        return 'consonant'
df.groupby(get_letter_type, axis=1).groups

{'consonant': Index(['B', 'C', 'D'], dtype='object'),
 'vowel': Index(['A'], dtype='object')}

In [39]:
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [41]:
[get_letter_type(e) for e in df.columns]

['vowel', 'consonant', 'consonant', 'consonant']

In [42]:
for name, group in df.groupby(get_letter_type, axis=1):
    print(name)
    print(group)

consonant
       B         C         D
0    one  1.764052 -0.103219
1    one  0.400157  0.410599
2    two  0.978738  0.144044
3  three  2.240893  1.454274
4    two  1.867558  0.761038
5    two -0.977278  0.121675
6    one  0.950088  0.443863
7  three -0.151357  0.333674
vowel
     A
0  foo
1  bar
2  foo
3  bar
4  foo
5  bar
6  foo
7  foo


### 2.5 GroupBy external variables

In [45]:
# group by whether an obs is at even position
isEvenIx = (1 + np.arange(len(df.index))) % 2 == 0
isEvenIx

array([False,  True, False,  True, False,  True, False,  True], dtype=bool)

In [46]:
for name, group in df.groupby(isEvenIx):
    print(name)
    print(group)

False
     A    B         C         D
0  foo  one  1.764052 -0.103219
2  foo  two  0.978738  0.144044
4  foo  two  1.867558  0.761038
6  foo  one  0.950088  0.443863
True
     A      B         C         D
1  bar    one  0.400157  0.410599
3  bar  three  2.240893  1.454274
5  bar    two -0.977278  0.121675
7  foo  three -0.151357  0.333674


In [49]:
# Average age in each interval
age = pd.Series(np.random.randint(1,100,50))
factor = pd.qcut(age, 4)
pd.DataFrame({'age':age, 'factor':factor})

,age,factor
0,82,"(71.75, 99.0]"
1,51,"(49.0, 71.75]"
2,28,"(21.25, 49.0]"
3,15,"(0.999, 21.25]"
4,42,"(21.25, 49.0]"
5,59,"(49.0, 71.75]"
6,66,"(49.0, 71.75]"
7,37,"(21.25, 49.0]"
8,11,"(0.999, 21.25]"
9,87,"(71.75, 99.0]"


In [50]:
age.groupby(factor).mean()

(0.999, 21.25]    10.384615
(21.25, 49.0]     36.416667
(49.0, 71.75]     59.916667
(71.75, 99.0]     84.769231
dtype: float64

## 3. Aggregation
### 3.1 aggregate (agg) method

In [52]:
df

,A,B,C,D
0,foo,one,1.764052,-0.103219
1,bar,one,0.400157,0.410599
2,foo,two,0.978738,0.144044
3,bar,three,2.240893,1.454274
4,foo,two,1.867558,0.761038
5,bar,two,-0.977278,0.121675
6,foo,one,0.950088,0.443863
7,foo,three,-0.151357,0.333674


In [54]:
# The sum of obs over groups indicated in column A
grouped = df.groupby('A')
grouped.agg(np.mean)

,C,D
A,,
bar,0.554591,0.662182
foo,1.081816,0.315880


In [57]:
# The mean of obs over groups indicated in columns A&B
grouped = df.groupby(['A','B'])
grouped.aggregate(np.mean)

,A,B,C,D
0,bar,one,0.400157,0.410599
1,bar,three,2.240893,1.454274
2,bar,two,-0.977278,0.121675
3,foo,one,1.357070,0.170322
4,foo,three,-0.151357,0.333674
5,foo,two,1.423148,0.452541


In [56]:
# Having group names as columns rather than index
grouped = df.groupby(['A', 'B'], as_index=False)
grouped.aggregate(np.mean)

,A,B,C,D
0,bar,one,0.400157,0.410599
1,bar,three,2.240893,1.454274
2,bar,two,-0.977278,0.121675
3,foo,one,1.357070,0.170322
4,foo,three,-0.151357,0.333674
5,foo,two,1.423148,0.452541


### 3.2 Some common shorthands

In [58]:
# Dispatching to instance methods
# currently sum, mean, std, and sem are Cython-optimized (fast)
df.groupby('A').sum()

,C,D
A,,
bar,1.663773,1.986547
foo,5.409080,1.579400


In [59]:
df.groupby('A').agg(np.sum)

,C,D
A,,
bar,1.663773,1.986547
foo,5.409080,1.579400


### 3.3 Applying multiple functions

In [60]:
grouped = df.groupby('A')

In [62]:
# On all applicable non-key columns (A is key, not applicable to B)
grouped.agg([np.sum, np.mean, np.std])

C                             D                    
          sum      mean       std       sum      mean       std
A                                                              
bar  1.663773  0.554591  1.614634  1.986547  0.662182  0.701017
foo  5.409080  1.081816  0.811100  1.579400  0.315880  0.324050

In [63]:
# On a given column
grouped['D'].agg([np.sum, np.mean, np.std])

,sum,mean,std
A,,,
bar,1.986547,0.662182,0.701017
foo,1.579400,0.315880,0.324050


In [64]:
# provide names for the aggregated columns
grouped.agg([np.sum, np.mean, np.std]).rename(columns={'sum':'total','mean':'avg'})

C                             D                    
        total       avg       std     total       avg       std
A                                                              
bar  1.663773  0.554591  1.614634  1.986547  0.662182  0.701017
foo  5.409080  1.081816  0.811100  1.579400  0.315880  0.324050

In [66]:
# Apply different functions to different columns
grouped.agg({'C' : np.sum, 'D' : lambda x: np.std(x, ddof=1)})

,C,D
A,,
bar,1.663773,0.701017
foo,5.409080,0.324050


In [67]:
def std1(x): 
    return np.std(x, ddof=1)
grouped.agg({'C' : np.sum, 'D' : std1})

,C,D
A,,
bar,1.663773,0.701017
foo,5.409080,0.324050


## 4. Transformation

### 4.1 Standardization

In [68]:
# data generation
np.random.seed(0)
index = pd.date_range('2015/01/01', periods=24, freq='M')
df3 = pd.DataFrame({'A':np.random.normal(0.5,2,len(index)), 'B':np.random.normal(-0.5,1,len(index))}, index)
df3

,A,B
2015-01-31,4.028105,1.769755
2015-02-28,1.300314,-1.954366
2015-03-31,2.457476,-0.454241
2015-04-30,4.981786,-0.687184
2015-05-31,4.235116,1.032779
2015-06-30,-1.454556,0.969359
2015-07-31,2.400177,-0.345053
2015-08-31,0.197286,-0.121837
2015-09-30,0.293562,-1.387786
2015-10-31,1.321197,-2.480796


In [69]:
# apply zscore standardization on a yearly basis 
year = lambda x: x.year
zscore = lambda x: (x - x.mean()) / x.std()
transformed = df3.groupby(year).transform(zscore)
transformed

,A,B
2015-01-31,1.054047,1.762992
2015-02-28,-0.361146,-1.257062
2015-03-31,0.239196,-0.040545
2015-04-30,1.548822,-0.229448
2015-05-31,1.161445,1.165346
2015-06-30,-1.790388,1.113916
2015-07-31,0.209469,0.048001
2015-08-31,-0.933403,0.229016
2015-09-30,-0.883454,-0.797597
2015-10-31,-0.350312,-1.683968


Exercise: check if the mean and std are indeed 0 and 1, respectively, for each year

In [72]:
transformed.describe()

,A,B
count,2.400000e+01,2.400000e+01
mean,1.110223e-16,-2.312965e-17
std,9.780193e-01,9.780193e-01
min,-2.472695e+00,-1.683968e+00
25%,-6.587243e-01,-7.642527e-01
50%,2.243323e-01,-1.575397e-01
75%,6.873946e-01,8.727087e-01
max,1.548822e+00,1.785299e+00


In [73]:
transformed.agg([np.mean, np.std])

,A,B
mean,1.110223e-16,-2.312965e-17
std,9.780193e-01,9.780193e-01


### 4.2 Filling missing values

In [74]:
# suppose some months are missing
df3.iloc[[3,19,20]] = None
df3

,A,B
2015-01-31,4.028105,1.769755
2015-02-28,1.300314,-1.954366
2015-03-31,2.457476,-0.454241
2015-04-30,NaN,NaN
2015-05-31,4.235116,1.032779
2015-06-30,-1.454556,0.969359
2015-07-31,2.400177,-0.345053
2015-08-31,0.197286,-0.121837
2015-09-30,0.293562,-1.387786
2015-10-31,1.321197,-2.480796


In [75]:
df3.groupby(year).transform(lambda x: x.fillna(x.mean()))

,A,B
2015-01-31,4.028105,1.769755
2015-02-28,1.300314,-1.954366
2015-03-31,2.457476,-0.454241
2015-04-30,1.725028,-0.378523
2015-05-31,4.235116,1.032779
2015-06-30,-1.454556,0.969359
2015-07-31,2.400177,-0.345053
2015-08-31,0.197286,-0.121837
2015-09-30,0.293562,-1.387786
2015-10-31,1.321197,-2.480796


In [76]:
df3.groupby(year).mean()

,A,B
2015,1.725028,-0.378523
2016,1.307626,-0.834518


### 4.3 Transform vs. Aggregate

In [78]:
# transform always return the same number of obs
df3.groupby(year).transform(lambda x: x.max() - x.min())

,A,B
2015-01-31,5.689672,4.250551
2015-02-28,5.689672,4.250551
2015-03-31,5.689672,4.250551
2015-04-30,5.689672,4.250551
2015-05-31,5.689672,4.250551
2015-06-30,5.689672,4.250551
2015-07-31,5.689672,4.250551
2015-08-31,5.689672,4.250551
2015-09-30,5.689672,4.250551
2015-10-31,5.689672,4.250551


In [79]:
df3.groupby(year).agg(lambda x: x.max() - x.min())

,A,B
2015,5.689672,4.250551
2016,4.472488,2.936561


## 5. Filtration
The filter method returns a subset of the original object.

In [80]:
# Focus on groups with group sum greater than 2
sf = pd.Series([1, 1, 2, 3, 3, 3])
sf.groupby(sf).sum()

1    2
2    2
3    9
dtype: int64

In [81]:
sf.groupby(sf).filter(lambda x: x.sum() > 2)

3    3
4    3
5    3
dtype: int64

In [82]:
dff = pd.DataFrame({'A': np.arange(8), 'B': list('aabbbbcc')})
dff

,A,B
0,0,a
1,1,a
2,2,b
3,3,b
4,4,b
5,5,b
6,6,c
7,7,c


In [83]:
# remove obs in groups with less than or equal to 2 obs
dff.groupby('B').filter(lambda x: len(x) > 2)

,A,B
2,2,b
3,3,b
4,4,b
5,5,b


In [84]:
# keep filtered obs as missing
dff.groupby('B').filter(lambda x: len(x) > 2, dropna=False)

,A,B
0,NaN,NaN
1,NaN,NaN
2,2.0,b
3,3.0,b
4,4.0,b
5,5.0,b
6,NaN,NaN
7,NaN,NaN


In [91]:
# multiple columns case
dff['C'] = np.arange(8)
dff.groupby('B').filter(lambda x: np.mean(x['C']) > 2)

,A,B,C
2,2,b,2
3,3,b,3
4,4,b,4
5,5,b,5
6,6,c,6
7,7,c,7


## 6. Flexible apply (Vectorized Operations)
A more general method for data aggregation, including those can't be conveniently done by "transform" and "aggregate"

In [92]:
# compute the demeaned version of column C while keeping the original
def f(group):
    return pd.DataFrame({'original' : group, 'demeaned' : group - group.mean()})
df.groupby('A')['C'].apply(f)

,demeaned,original
0,0.682236,1.764052
1,-0.154434,0.400157
2,-0.103078,0.978738
3,1.686302,2.240893
4,0.785742,1.867558
5,-1.531869,-0.977278
6,-0.131727,0.950088
7,-1.233173,-0.151357


In [95]:
# summary stats of each group
df.groupby('A').apply(pd.Series.describe).unstack()

C                                                              \
    count      mean       std       min       25%       50%       75%   
A                                                                       
bar   3.0  0.554591  1.614634 -0.977278 -0.288560  0.400157  1.320525   
foo   5.0  1.081816  0.811100 -0.151357  0.950088  0.978738  1.764052   

                  D                                                    \
          max count      mean       std       min       25%       50%   
A                                                                       
bar  2.240893   3.0  0.662182  0.701017  0.121675  0.266137  0.410599   
foo  1.867558   5.0  0.315880  0.324050 -0.103219  0.144044  0.333674   

                         
          75%       max  
A                        
bar  0.932436  1.454274  
foo  0.443863  0.761038

In [94]:
# There is a shorthand, and slightly more readable
# What is done after the calling the apply method?
df.groupby('A').describe()

C                                                              \
    count      mean       std       min       25%       50%       75%   
A                                                                       
bar   3.0  0.554591  1.614634 -0.977278 -0.288560  0.400157  1.320525   
foo   5.0  1.081816  0.811100 -0.151357  0.950088  0.978738  1.764052   

                  D                                                    \
          max count      mean       std       min       25%       50%   
A                                                                       
bar  2.240893   3.0  0.662182  0.701017  0.121675  0.266137  0.410599   
foo  1.867558   5.0  0.315880  0.324050 -0.103219  0.144044  0.333674   

                         
          75%       max  
A                        
bar  0.932436  1.454274  
foo  0.443863  0.761038

Exercise: generate a DataFrame for the following series, column1=s, column2=s**2, column3=log(s)

In [ ]:
np.random.seed(928)
s = pd.Series(np.random.randint(1,10,6))
s